In [1]:
#importing_libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
#create_data_loader
batch_size = 64
learning_rate = 0.001
epochs = 5

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  
])

train_data = datasets.MNIST(root="data", train=True, download=True, transform=transform)
test_data = datasets.MNIST(root="data", train=False, download=True, transform=transform)


train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)


In [3]:
#Building_and_defining_the_model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
       
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        
        self.relu1 = nn.ReLU()
      
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        self.relu2 = nn.ReLU()
        
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        
        self.dropout = nn.Dropout(p=0.25)  

        
        self.fc1 = nn.Linear(in_features=7 * 7 * 64, out_features=10)  # 7x7 output size after 2 max pooling layers


    def forward(self, x):
        
        x = self.relu1(self.conv1(x))
        x = self.pool(x)
        x = self.relu2(self.conv2(x))
        x = self.pool2(x)

        
        x = self.dropout(x)

       
        x = x.view(-1, x.size(1) * x.size(2) * x.size(3))
        x = self.fc1(x)
        
        output = F.log_softmax(x, dim=1)
        return output

In [4]:
model = CNN()

In [5]:
#define_loss_and_optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
#training
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and parameter update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


Epoch [1/5], Step [100/938], Loss: 0.1598
Epoch [1/5], Step [200/938], Loss: 0.1283
Epoch [1/5], Step [300/938], Loss: 0.0329
Epoch [1/5], Step [400/938], Loss: 0.1601
Epoch [1/5], Step [500/938], Loss: 0.0350
Epoch [1/5], Step [600/938], Loss: 0.0233
Epoch [1/5], Step [700/938], Loss: 0.1144
Epoch [1/5], Step [800/938], Loss: 0.0594
Epoch [1/5], Step [900/938], Loss: 0.0339
Epoch [2/5], Step [100/938], Loss: 0.1440
Epoch [2/5], Step [200/938], Loss: 0.0525
Epoch [2/5], Step [300/938], Loss: 0.0267
Epoch [2/5], Step [400/938], Loss: 0.0617
Epoch [2/5], Step [500/938], Loss: 0.1157
Epoch [2/5], Step [600/938], Loss: 0.0363
Epoch [2/5], Step [700/938], Loss: 0.0102
Epoch [2/5], Step [800/938], Loss: 0.1241
Epoch [2/5], Step [900/938], Loss: 0.0194
Epoch [3/5], Step [100/938], Loss: 0.0405
Epoch [3/5], Step [200/938], Loss: 0.0112
Epoch [3/5], Step [300/938], Loss: 0.0037
Epoch [3/5], Step [400/938], Loss: 0.1139
Epoch [3/5], Step [500/938], Loss: 0.0421
Epoch [3/5], Step [600/938], Loss:

In [7]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')


Accuracy of the model on the test images: 98.85%


In [8]:
torch.cuda.empty_cache()

In [9]:
from torchsummary import summary

model = CNN()
summary(model, (1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 32, 28, 28]          320
├─ReLU: 1-2                              [-1, 32, 28, 28]          --
├─MaxPool2d: 1-3                         [-1, 32, 14, 14]          --
├─Conv2d: 1-4                            [-1, 64, 14, 14]          18,496
├─ReLU: 1-5                              [-1, 64, 14, 14]          --
├─MaxPool2d: 1-6                         [-1, 64, 7, 7]            --
├─Dropout: 1-7                           [-1, 64, 7, 7]            --
├─Linear: 1-8                            [-1, 10]                  31,370
Total params: 50,186
Trainable params: 50,186
Non-trainable params: 0
Total mult-adds (M): 3.87
Input size (MB): 0.00
Forward/backward pass size (MB): 0.29
Params size (MB): 0.19
Estimated Total Size (MB): 0.48


Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 32, 28, 28]          320
├─ReLU: 1-2                              [-1, 32, 28, 28]          --
├─MaxPool2d: 1-3                         [-1, 32, 14, 14]          --
├─Conv2d: 1-4                            [-1, 64, 14, 14]          18,496
├─ReLU: 1-5                              [-1, 64, 14, 14]          --
├─MaxPool2d: 1-6                         [-1, 64, 7, 7]            --
├─Dropout: 1-7                           [-1, 64, 7, 7]            --
├─Linear: 1-8                            [-1, 10]                  31,370
Total params: 50,186
Trainable params: 50,186
Non-trainable params: 0
Total mult-adds (M): 3.87
Input size (MB): 0.00
Forward/backward pass size (MB): 0.29
Params size (MB): 0.19
Estimated Total Size (MB): 0.48

In [10]:
torch.save(model.state_dict(), "Assignment.pt")